In [1]:
from pathlib import Path
home = str(Path.home()/'work')
import sys
#sys.path.insert(1, home)
sys.path.append(home)

from numa.equation_systems import simple_gaussian_elimination, back_substitution, lu_factorization, pa_lu_factorization, jacobi

import numpy as np

# Naive Gaussian Elimination

Elementary row operations:
1. Switch two rows.
2. Multiply a row by a nonzero number.
3. Replace a row by any multiple of another row added to it.

xref [1.3 Gaussian Elimination ](https://math.libretexts.org/Bookshelves/Linear_Algebra/A_First_Course_in_Linear_Algebra_(Kuttler)/01%3A_Systems_of_Equations/1.03%3A_Gaussian_Elimination).


Let's start with an example,
$$
\begin{bmatrix}
1 & 1  \\
3 & -4 \\
\end{bmatrix}
\begin{bmatrix}
x_1  \\
x_2 \\
\end{bmatrix}
=
\begin{bmatrix}
3  \\
2 \\
\end{bmatrix}
$$

or using a different notation,

$$
\begin{bmatrix}
1 & 1  \\
3 & -4 \\
\end{bmatrix}
\begin{bmatrix}
x  \\
y \\
\end{bmatrix}
=
\begin{bmatrix}
3  \\
2 \\
\end{bmatrix}
$$

The augmented matrix form of this system of equations looks as follows
(we included the Gaussian elimination step to get the solution):
$$
\left[\begin{array}{@{}cc|c@{}}
1 & 1  & 3 \\
3 & -4 & 2
\end{array}\right]
\rightarrow
\left[\begin{array}{@{}cc|c@{}}
1 & 1  & 3 \\
0 & -7 & 7
\end{array}\right]
$$

The solution is then $(x, y) = (2, 1)$.

The implementation of the naive gaussian elimination follows the same mechanics:

1. Iterate through all columns (since we want to eliminate each element from below the diagonal going column by column.
2. If $a_{jj} = 0$, then raise an execption because we will always subtract a ratio of the pivot from the next equation.<br>
3. In our example above, 1 was our first pivot, and we subtracted the ratio of the item right below the pivot, 3, and the pivot, 1, - $3/1$ - from the item right below for each entry in a row.
4. In the code, we did not bother to zero out the element before the pivot because why waste an operation since Gaussian elimination results in all elements below the diagonal being zeros. 

In [35]:
# Let's now solve our system with our home-made numa implementation.
a = np.array([[1., 1.], [3., -4.]])
b = np.array([[3], [2]]).flatten()  # We use flatten() because we decided to represent a column vector litarally.

In [36]:
a

array([[ 1.,  1.],
       [ 3., -4.]])

In [37]:
b

array([3, 2])

In [38]:
# sol_a and sol_b are the a matrix and the b column vector corresponding to the last step in the augmented matrix.
sol_a, sol_b = simple_gaussian_elimination(a, b)

In [39]:
# Remember that we avoided zeroing out the elements below the diagonal just to save on a couple operations.
sol_a

array([[ 1.,  1.],
       [ 3., -7.]])

In [40]:
sol_b

array([ 3., -7.])

In [41]:
# Obtain the solution.
c = back_substitution(sol_a, sol_b)
c

array([2., 1.])

# LU Factorization

Here we brack the **coefficient matrix** $A$ into a product of a lower triangular matrix $L$ and an upper triangular
matrix $U$.

1. An $m \times n$ matrix $L$ is lower triangular if the entries satisfy $l_{ij} = 0$ for $i < j$.
2. An $m \times n$ matrix $U$ is upper triangular if the entries satisfy $u_{ij} = 0$ for $i > j$.

The trick is to
1. Keep 1's in the diagonal.
2. Store the result of the Gaussian elimination in $U$ and we store the multipliers (the ratio of the element right below the pivot and the pivot) we use for elimination in $L$.

$$
LU =
\begin{bmatrix}
1 & 0  \\
3 & 1 \\
\end{bmatrix}
\begin{bmatrix}
1 & 1  \\
0 & -7 \\
\end{bmatrix}
=
\begin{bmatrix}
1 & 1  \\
3 & -4 \\
\end{bmatrix}
=
A
$$

Another example:
$$
\begin{bmatrix}
1  & 2 & -1 \\
2  & 1 & -2 \\
-3 & 1 & 1
\end{bmatrix}
$$

This can be LU factorized into

$$
LU =
\begin{bmatrix}
1  & 0    & 0 \\
2  & 1    & 0 \\
-3 & -7/3 & 1
\end{bmatrix}
\begin{bmatrix}
1 & 2  & -1 \\
0 & -3 & 0 \\
0 & 0  & -2
\end{bmatrix}
=
\begin{bmatrix}
1  & 2 & -1 \\
2  & 1 & -2 \\
-3 & 1 & 1
\end{bmatrix}
= A
$$

Again, $U$ is the result of the Gaussian elimination and $L$ contains the multipliers we used throughout the elimination process.
So you can see that
1. to get rid of $a_{21} = 2$, we subtracted 2 times row 1 from row 2;
2. to eliminate $a_{31} = -3$, we subtracted -3 times row 1 from row 3;
3. to eliminate $a_{32}$, we subtracted -7/3 times row 2 from row 3.

And all those multiplers are stored as they were used.

Right away we can see the use of the LU factorization: we can resuse $L$ and $U$ to solve the system of equations for any $b$!

To get a solution, recall that we now have $Ax = LUx = b$.
If just so happens that if we define an "auxiliary" vector $c = Ux$.
Then,

1. Solve $Lc = b$ for $c$.
2. Solve $Ux = b$ for $x$.

In [43]:
a = np.array([[1, 2, -1], [2, 1, -2], [-3, 1, 1]])
a

array([[ 1,  2, -1],
       [ 2,  1, -2],
       [-3,  1,  1]])

In [44]:
L, U = lu_factorization(a)

In [45]:
L

array([[ 1.        ,  0.        ,  0.        ],
       [ 2.        ,  1.        ,  0.        ],
       [-3.        , -2.33333333,  1.        ]])

In [46]:
U

array([[ 1.,  2., -1.],
       [ 0., -3.,  0.],
       [ 0.,  0., -2.]])

Let $L_{ij}(-c)$ denote a lower triangular matrixwhose only nonzero entries are 1s in the diagonal and $-c$ in the $(i, j)$
position.
Then $A \rightarrow L_{ij}(-c)$ represents the operation "subtracting $c$ times row $j$ from row $i$".

For example, if we multiply a generic $3\times 3$ matrix A by $L_{21}(-c)$ (subtracting $c$ times row 1 from row 2)
$$
\begin{bmatrix}
1  & 0 & 0 \\
-c & 1 & 0 \\
0  & 0 & 1
\end{bmatrix}
\begin{bmatrix}
a_{11} & a_{12} & a_{13} \\
a_{21} & a_{22} & a_{23} \\
a_{31} & a_{32} & a_{33} \\
\end{bmatrix}
=
\begin{bmatrix}
a_{11} & a_{12} & a_{13} \\
a_{21} - ca_{11} & a_{22} - ca_{12} & a_{23} - ca_{13} \\
a_{31} & a_{32} & a_{33} \\
\end{bmatrix}
$$

$L_{ij}(-c)^{-1} = L_{ij}(c)$, for example

$$
\begin{bmatrix}
1  & 0 & 0 \\
-c & 1 & 0 \\
0  & 0 & 1
\end{bmatrix}^{-1}
=
\begin{bmatrix}
1  & 0 & 0 \\
c & 1 & 0 \\
0  & 0 & 1
\end{bmatrix}
$$

LU factorization also holds because this matrix product equation holds:
$$
\begin{bmatrix}
1   & 0 & 0 \\
c_1 & 1 & 0 \\
0   & 0 & 1
\end{bmatrix}
\begin{bmatrix}
1   & 0 & 0 \\
0   & 1 & 0 \\
c_2 & 0 & 1
\end{bmatrix}
\begin{bmatrix}
1 & 0   & 0 \\
0 & 1   & 0 \\
0 & c_3 & 1
\end{bmatrix}
=
\begin{bmatrix}
1   & 0   & 0 \\
c_1 & 1   & 0 \\
c_2 & c_3 & 1
\end{bmatrix}
$$

# References

1. [A First Course in Linear Algebra (Kuttler)](https://math.libretexts.org/Bookshelves/Linear_Algebra/A_First_Course_in_Linear_Algebra_(Kuttler)/01%3A_Systems_of_Equations/1.03%3A_Gaussian_Elimination)
2. Numerical Analysis by Timothy Sauer, chapter 2.

## Span and Rank


The **span** of a set $S$ of vectors (from a vector space), is defined as the set of all linear combinations of the vectors in $S$.
For example, two linearly independent vectors span a plane.
The span can be characterized either as the intersection of all linear subspaces that contain S, or as the smallest subspace containing S.

Given a vector space $V$ over a field $F$ (i.e., $\mathbb{R}$ or $\mathbb{C}$), the span of a set $S$ of vectors (not necessarily finite) is defined to be the intersection $W$ of all subspaces of $V$ that contain $S$.
$W$ is referred to as the subspace spanned by $S$, or by the vectors in $S$.
Conversely, $S$ is called a spanning set of $W$, and we say that $S$ spans $W$.

$$
span(S) = \Big\{
    \sum_{i\in\mathbb{N}} \lambda_i \vec{v}_i   \quad \Big| \quad  \lambda \in F, \vec{v}\in S
\Big\}
$$

xref: [wikipedia: linear span](https://en.wikipedia.org/wiki/Linear_span).


A good way to help make better sense of the span of a matrix is to use it to understand the rank of a mtrix.

The **rank** of a matrix $A$ is the dimension of the vector space generated (or spanned) by its columns.
This corresponds to the maximal number of linearly independent **columns** of $A$.
This, in turn, is identical to the dimension of the vector space spanned by its rows.


xref: [wikipedia: rank](https://en.wikipedia.org/wiki/Rank_(linear_algebra))

### Other properties

1. Recall that the determinant of a matrix is zero if any of its rows or columns are not linearly independent. Conversely, if the determinant is non-zero, that implies that a square matrix is invertible and that its columns or rows span $\mathbb{R}^m$.

## Basis

Building on top of the idea of a spaning set, a set $B$ of vectors in a vector space $V$ is called a basis if every element of $V$ may be written in a unique way as a finite linear combination of elements of $B$.

Equivalently, a set $B$ is a basis if its elements are linearly independent and every element of $V$ is a linear combination of elements of $B$. In other words, **a basis is a linearly independent spanning set**.

A basis $B$ of a vector space $V$ over a field $F$ (such as the real numbers $\mathbb{R}$ or the complex numbers $\mathbb{C}$) is a linearly independent subset of $V$ that spans $V$.
This means that a subset $B$ of $V$ is a basis if it satisfies the two following conditions:

1. Linear independence

for every finite subset $\{\mathbf{v}_{1}, \dotsc ,\mathbf{v}_{m} \}$ of $B$, if
$c_{1}\mathbf{v}_{1} + \cdots + c_{m}\mathbf{v} _{m} = \mathbf{0}$
for some $c_{1}, \dotsc ,c_{m}$ in $F$, then $c_{1} = \cdots = c_{m} = 0$.

2. Spanning property

for every vector $\mathbf{v}$ in $V$, one can choose $a_{1}, \dotsc ,a_{n}$ in $F$ and
$\mathbf{v}_{1}, \dotsc , \mathbf{v}_{n}$ in $B$ such that
$\mathbf{v} = a_{1} \mathbf {v}_{1} + \cdots + a_{n}\mathbf{v}_{n}$.

xref: [wikipedia: basis](https://en.wikipedia.org/wiki/Basis_(linear_algebra))

Example: this is a non-trivial interesting example. The standard basis for $2x2$ matrices is formed by
$$
\mathbf{e}_{11}
=
\begin{pmatrix}
1 & 0 \\
0 & 0
\end{pmatrix}
$$

$$
\mathbf{e}_{12}
=
\begin{pmatrix}
0 & 1 \\
0 & 0
\end{pmatrix}
$$

$$
\mathbf{e}_{21}
=
\begin{pmatrix}
0 & 0 \\
1 & 0
\end{pmatrix}
$$

$$
\mathbf{e}_{22}
=
\begin{pmatrix}
0 & 0 \\
0 & 1
\end{pmatrix}
$$

### Other Properties

1. If $A$ is an $m\times m$ matrix and it has eigenvectors that form a basis for $\mathbb{R}^m$, then $A$ is **similar** to a **diagonal** matrix, and $A$ is called **diagonalizable**. xref: Numerical Analysis by Sauer, A.3.

One additional property that builds upon this is that if we have $Ax_i = \lambda x_i$ for $i = 1, \dots, m$, and we define a matrix out of the eigenvectors as

$$
S = \left[ \quad x_1 \quad | \quad \dots \quad | \quad x_m \quad \right]
$$

Making an $m\times m$ matrix, then

$$
AS = S
\begin{pmatrix}
\lambda_1 &        &           \\
          & \ddots &           \\
          &        & \lambda_m
\end{pmatrix}
$$

From here we can also see that
* $S$ is invertible since its columns span $\mathbb{R}^m$.
* $A$ is similar to the diagonal column containing its eigenvalues.

## Similar Matrices


The matrices $m \times m$ $A_1$ and $A_2$ are **similar**, denoted $A_1 \sim A_2$, if there exists an invertible matrix $S$ such that

$$
A_1 = S A_2 S^{-1}
$$

1. Similar matrices have identical eigenvalues, because their characteristic polynomials are the same.

$$
A_1 - \lambda I = S A S^{-1} - \lambda I = S \left( A_2 - \lambda I \right) S^{-1}
$$

implies
$$
\det(A_1 - \lambda I) = \det(S) \det(A_2 - \lambda I) \det(S^{-1}) = \det(A_2 - \lambda I)
$$

xref:
1. Numerical Anlysis by Sauer A.3
2. Lemma 7.1.1 in A First Course in Linear Algebra (Kuttler)

## Symmetric and Herminitan Matrices

Symmetric and Hermitian matrices have
1. All real eigenvalues
2. All eigenvectors are orthogonal to one another
3. Their eigenvectors span the underlying space
4. The set of unit eigenvectors of these matrices are an orthonormal set for the underlying space

## Spectral Radius

The **spectral radisu** of a square matrix is the maximum magnitude of its eigenvalues.